TODO : 

CLean Spotify Code

deezer: when searching for deezertrackid, update the library_spotify with deezertrackid

add playlist spotify

# SPOTIFY

#### Connect and Get all tracks

In [3]:
import spotipy
import pandas as pd

from pprint import pprint

from spotipy.oauth2 import SpotifyOAuth

## LOAD ENVIRONMENT VARIABLES : 
from dotenv import load_dotenv
import os
load_dotenv()


spotify = spotipy.Spotify(auth_manager=spotipy.SpotifyOAuth())
me = spotify.me()
pprint(me)

{'display_name': 'Thealamenthe',
 'external_urls': {'spotify': 'https://open.spotify.com/user/jbelguiz'},
 'followers': {'href': None, 'total': 17},
 'href': 'https://api.spotify.com/v1/users/jbelguiz',
 'id': 'jbelguiz',
 'images': [],
 'type': 'user',
 'uri': 'spotify:user:jbelguiz'}


In [3]:
# Shows a user's saved tracks (need to be authenticated via oauth) 
# 
# MAIN FUNCTION IN USE !
# add track id for every track 


scope = 'user-library-read'

tracklist = []
artistlist = []
albumlist = []
trackid = []

def show_tracks(results):
    for item in results['items']:
        track = item['track']
        #print("%32.32s %s -id %s --album : %s" % (track['artists'][0]['name'], track['name'],track['id'],track['album']['name']))
        tracklist.append(track['name'])
        artistlist.append(track['artists'][0]['name'])
        albumlist.append(track['album']['name'])
        trackid.append(track['id'])


sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope))

results = sp.current_user_saved_tracks()
show_tracks(results)

while results['next']:
    results = sp.next(results)
    show_tracks(results)

In [ ]:
results


In [4]:
dictionary = {
    'Artists': artistlist, 
    'Tracks': tracklist, 
    'Albums': albumlist,
    'TrackID': trackid
    }
df_library_spotify = pd.DataFrame(dictionary)
df_library_spotify

,Artists,Tracks,Albums,TrackID
0,Rene & Angela,I Love You More - Remastered,Classic Masters,14u1EaefnEWqLnuccfTLLg
1,"Joaquin ""Joe"" Claussell",Je Ka Jo - Demo Mix,Turntables on the Hudson: Twenty Years of Good...,2xzIpb5d2Cg34ZrPcEz1av
2,Francis Harris,Minor Forms - Joaquin Joe Claussell's Cosmic A...,Trivial Occupations Remixes,4BAKfGTTkkIrykZFFFaTKt
3,RuPaul,Supermodel (You Better Work),Supermodel of the World,1AAwAM9IjTlvi8x10kc2I1
4,Oumou Sangaré,Diaraby Nene,Moussolou,56JK50k6fKB4Yc8xOo18uN
...,...,...,...,...
4558,S-Crew,Une étoile,Destins liés,1bAHsggx8whuj34dybkvb9
4559,S-Crew,Sereins,Destins liés,55JShyK57fen0oVgslioHc
4560,S-Crew,Nés pour mener,Destins liés,6CrJRAtHIQM8PRB0EtyKdM
4561,Nekfeu,Jusqu'au bout (Creed),Destins liés,5mWqKF08KDoqD69eZWrEgt


In [ ]:
df_library_spotify.to_excel('test.xlsx',index =False)

#### Get all playlist and their tracks

In [4]:
# playlist track id 
def trackID_from_playlistID (playlistID): 
    ''' Return a list of track ID for a given PlaylistID
    '''
    pl_id = 'spotify:playlist:'+playlistID
    offset = 0
    trackIDS=[]
    tracklist = []
    artistlist = []
    albumlist = []
    while True:
        response = sp.playlist_items(pl_id,
                                    offset=offset,
                                    fields='items.track.id,items.track.album.name,items.track.name,items.track.artists.name,total',
                                    additional_types=['track'])
        
        if len(response['items']) == 0:
            break
        
        pprint(response)
        offset = offset + len(response['items'])

        for track in response['items']:
            trackIDS.append(track['track']['id'])
            tracklist.append(track['track']['name'])
            artistlist.append(track['track']['artists'][0]['name'])
            albumlist.append(track['track']['album']['name'])

    if response['total']==len(trackIDS):
        return trackIDS,tracklist,artistlist,albumlist
    else : 
        return None

# get playlist id : 

scope = 'playlist-read-private'
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope))

playlists = sp.current_user_playlists(limit=50)

playlist_name = []
playlist_id = []
for i, item in enumerate(playlists['items']):
    playlist_id.append(item['id'])
    playlist_name.append(item['name'])
    print("%d %s %s" % (i, item['name'], item['id']))

# get tracks from playlists

df_playlist = pd.DataFrame(columns=['PlaylistName','TrackID','tracklist','artistlist','albumlist'])

for i in range(len(playlist_id)):
    # add one playlist
    df = pd.DataFrame(columns=['PlaylistName','TrackID'])
    trackIDS,tracklist,artistlist,albumlist = trackID_from_playlistID(playlist_id[i])
    df['TrackID'] = trackIDS
    df['tracklist'] = tracklist
    df['artistlist'] = artistlist
    df['albumlist'] = albumlist
    df['PlaylistName'] = playlist_name[i]

    df_playlist = pd.concat([df_playlist,df],ignore_index=True)

df_playlist

[0426/160238.191464:ERROR:file_io_posix.cc(152)] open /home/julien/.config/BraveSoftware/Brave-Browser/Crash Reports/pending/d8a18e46-cf4d-43d9-b158-faa67a850eef.lock: File exists (17)


Opening in existing browser session.
0 Your Top Songs 2021 37i9dQZF1EUMDoJuT8yJsl
1 Your Top Songs 2020 37i9dQZF1EMe2T0EyvguLP
2 Italove Radio 37i9dQZF1E8NbdbX2F45JR
3 Rap instru 1dXfMuIRYsKGJQ8M0umDtL
4 Rap Workout 37i9dQZF1DX76t638V6CA8
5 Italia 1yST9FK3aQR3nz3wTVEswY
6 italian way 0BlDB5xOIN3sVZU8mxxxud
7 Your Top Songs 2018 37i9dQZF1EjoRhwNKmilGy
8 La Playlist de France Inter (officiel) 1IhrDzE8Z4fegSvS8uHmJM
9 Minimal Techno 5fIKyffSDYwk3Lsp1fu6Ix
10 So ->2020 3SLqjv6aiWJvQPU3AOiZaq
11 Liked from Radio 0WkVZnEDeEBmYODkNwInpM
12 Shs 0GJ4Jn4Aiey2cryR6Qknaq
13 Inspi 3fdk3ojDPHoo2goiKauFYZ
14 Instrumental Hip Hop / Chill Hip Hop / Jazzy 1dQQkNsgAcfAzO1KTvHOTR
15 Your Top Songs 2017 37i9dQZF1E9TRuLHu7N8Tt
16 Mes titres Shazam 1OTkYknLsOm6EA9KmvSfJ0
17 Romantic songs 20jq3psqbPdWpDBlrCGPP5
18 Spotify's Most Played All-Time [Updated Weekly] | Most Streamed | Top Played | 500Mil+ 2YRe7HRKNRvXdJBp9nXFza
19 Jazz 4CBhdnEFSnAFT0Cd8mfJ6L
20 Unexpected 3yE2LMbxKxT5DHOFvZowqY
21 Rap - Turn Up 7C

,PlaylistName,TrackID,tracklist,artistlist,albumlist
0,Your Top Songs 2021,3JJL91ilRV6fXhKqu0FGXs,HOT WIND BLOWS (feat. Lil Wayne),"Tyler, The Creator",CALL ME IF YOU GET LOST
1,Your Top Songs 2021,0BiK5BbYNFLb88CCOxBFJe,LUMBERJACK,"Tyler, The Creator",CALL ME IF YOU GET LOST
2,Your Top Songs 2021,2goHCrsDJaflEwQ930ef0o,MASSA,"Tyler, The Creator",CALL ME IF YOU GET LOST
3,Your Top Songs 2021,4ydmav4vl7hebadtfZtMrg,SIR BAUDELAIRE (feat. DJ Drama),"Tyler, The Creator",CALL ME IF YOU GET LOST
4,Your Top Songs 2021,13HsOwrwTA5HgLuoaDHncP,RUNITUP (feat. Teezo Touchdown),"Tyler, The Creator",CALL ME IF YOU GET LOST
...,...,...,...,...,...
5334,Sexy songs - R&B,2rWXpcM2lmlf2i7qCNaXHM,Break You Off,The Roots,Phrenology
5335,Sexy songs - R&B,4LkLfPYc3yjOta7Gk8FDRB,Think About Me,dvsn,Morning After
5336,Sexy songs - R&B,42QRyJd5x6e2TFR1S41g2p,Too Fast,Sonder,Too Fast
5337,Sexy songs - R&B,5MjAFPcmrR1DYCgZrXCN4d,Turnin' Me Up,BJ The Chicago Kid,In My Mind


In [3]:
playlistID='37i9dQZF1EUMDoJuT8yJsl'

pl_id = 'spotify:playlist:'+playlistID

scope = 'playlist-read-private'
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope))


response = sp.playlist_items(pl_id,
                                    offset=0,
                                    fields='items.track.id,items.track.album.name,items.track.name,items.track.artists.name,total',
                                    additional_types=['track'])

response

Opening in existing browser session.


{'items': [{'track': {'album': {'name': 'CALL ME IF YOU GET LOST'},
    'artists': [{'name': 'Tyler, The Creator'}, {'name': 'Lil Wayne'}],
    'id': '3JJL91ilRV6fXhKqu0FGXs',
    'name': 'HOT WIND BLOWS (feat. Lil Wayne)'}},
  {'track': {'album': {'name': 'CALL ME IF YOU GET LOST'},
    'artists': [{'name': 'Tyler, The Creator'}],
    'id': '0BiK5BbYNFLb88CCOxBFJe',
    'name': 'LUMBERJACK'}},
  {'track': {'album': {'name': 'CALL ME IF YOU GET LOST'},
    'artists': [{'name': 'Tyler, The Creator'}],
    'id': '2goHCrsDJaflEwQ930ef0o',
    'name': 'MASSA'}},
  {'track': {'album': {'name': 'CALL ME IF YOU GET LOST'},
    'artists': [{'name': 'Tyler, The Creator'}, {'name': 'DJ Drama'}],
    'id': '4ydmav4vl7hebadtfZtMrg',
    'name': 'SIR BAUDELAIRE (feat. DJ Drama)'}},
  {'track': {'album': {'name': 'CALL ME IF YOU GET LOST'},
    'artists': [{'name': 'Tyler, The Creator'}, {'name': 'Teezo Touchdown'}],
    'id': '13HsOwrwTA5HgLuoaDHncP',
    'name': 'RUNITUP (feat. Teezo Touchdown)'}}

#### Select Playlists to keep 


In [5]:
df_playlist.PlaylistName.unique()

# to delete : 

to_delete=['Rap Workout','Italove Radio','Rap instru','Rap Workout','italian way','La Playlist de France Inter (officiel)', 'Minimal Techno',
    'So ->2020','Instrumental Hip Hop / Chill Hip Hop / Jazzy',"Spotify's Most Played All-Time [Updated Weekly] | Most Streamed | Top Played | 500Mil+"]

df_playlist_tokeep = df_playlist[~df_playlist['PlaylistName'].isin(to_delete)]
df_playlist_tokeep.reset_index(drop=True,inplace=True)
df_playlist_tokeep

,PlaylistName,TrackID,tracklist,artistlist,albumlist
0,Your Top Songs 2021,3JJL91ilRV6fXhKqu0FGXs,HOT WIND BLOWS (feat. Lil Wayne),"Tyler, The Creator",CALL ME IF YOU GET LOST
1,Your Top Songs 2021,0BiK5BbYNFLb88CCOxBFJe,LUMBERJACK,"Tyler, The Creator",CALL ME IF YOU GET LOST
2,Your Top Songs 2021,2goHCrsDJaflEwQ930ef0o,MASSA,"Tyler, The Creator",CALL ME IF YOU GET LOST
3,Your Top Songs 2021,4ydmav4vl7hebadtfZtMrg,SIR BAUDELAIRE (feat. DJ Drama),"Tyler, The Creator",CALL ME IF YOU GET LOST
4,Your Top Songs 2021,13HsOwrwTA5HgLuoaDHncP,RUNITUP (feat. Teezo Touchdown),"Tyler, The Creator",CALL ME IF YOU GET LOST
...,...,...,...,...,...
732,Sexy songs - R&B,2rWXpcM2lmlf2i7qCNaXHM,Break You Off,The Roots,Phrenology
733,Sexy songs - R&B,4LkLfPYc3yjOta7Gk8FDRB,Think About Me,dvsn,Morning After
734,Sexy songs - R&B,42QRyJd5x6e2TFR1S41g2p,Too Fast,Sonder,Too Fast
735,Sexy songs - R&B,5MjAFPcmrR1DYCgZrXCN4d,Turnin' Me Up,BJ The Chicago Kid,In My Mind


#### Select from playlists tracks to add to library


In [7]:
# add all tracks from selected playlist to library 

In [20]:
# tracks ID with playlist name
albums

,PlaylistName,TrackID,tracklist,artistlist,albumlist
0,Your Top Songs 2021,3JJL91ilRV6fXhKqu0FGXs,HOT WIND BLOWS (feat. Lil Wayne),"Tyler, The Creator",CALL ME IF YOU GET LOST
1,Your Top Songs 2021,0BiK5BbYNFLb88CCOxBFJe,LUMBERJACK,"Tyler, The Creator",CALL ME IF YOU GET LOST
2,Your Top Songs 2021,2goHCrsDJaflEwQ930ef0o,MASSA,"Tyler, The Creator",CALL ME IF YOU GET LOST
3,Your Top Songs 2021,4ydmav4vl7hebadtfZtMrg,SIR BAUDELAIRE (feat. DJ Drama),"Tyler, The Creator",CALL ME IF YOU GET LOST
4,Your Top Songs 2021,13HsOwrwTA5HgLuoaDHncP,RUNITUP (feat. Teezo Touchdown),"Tyler, The Creator",CALL ME IF YOU GET LOST


In [5]:
#library
df_library_spotify.head()

,Artists,Tracks,Albums,TrackID
0,Rene & Angela,I Love You More - Remastered,Classic Masters,14u1EaefnEWqLnuccfTLLg
1,"Joaquin ""Joe"" Claussell",Je Ka Jo - Demo Mix,Turntables on the Hudson: Twenty Years of Good...,2xzIpb5d2Cg34ZrPcEz1av
2,Francis Harris,Minor Forms - Joaquin Joe Claussell's Cosmic A...,Trivial Occupations Remixes,4BAKfGTTkkIrykZFFFaTKt
3,RuPaul,Supermodel (You Better Work),Supermodel of the World,1AAwAM9IjTlvi8x10kc2I1
4,Oumou Sangaré,Diaraby Nene,Moussolou,56JK50k6fKB4Yc8xOo18uN


#### Get all albums

In [20]:
scope = 'user-library-read'

artists = []
albums = []



sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope))

results = sp.current_user_saved_albums()

totalAlbums = results['total']

while len(albums)<totalAlbums:
    for item in results['items']:
        albumname = item['album']['name']
        if len(item['album']['artists']) == 1 : 
            artistname = item['album']['artists'][0]['name']
        else :
            print(f' new PUTE for {albumname}')
            artistname = ""
            for i,val in enumerate(item['album']['artists']):
                artistname = artistname+ val['name'] + '/'
        
        #print(f'Album name : {albumname} by {artistname}')
        artists.append(artistname)
        albums.append(albumname)
    results = sp.next(results)

df_albums_spotify = pd.DataFrame({
    'Artists': artists, 
    'Albums': albums,
    })
df_albums_spotify

 new PUTE for What Kinda Music
 new PUTE for Silent Struggles
 new PUTE for Neverland
 new PUTE for Revenge Of The Dreamers III: Director's Cut
 new PUTE for Energy (with A$AP Rocky & Sabrina Claudio) (feat. Sabrina Claudio) [Sonny Fodera Remix]
 new PUTE for 82 92 (feat. Mac Miller)
 new PUTE for Pas du même monde (feat. Bigflo & Oli)
 new PUTE for Madvillainy
 new PUTE for The Midnight Hour
 new PUTE for La La Land (Original Motion Picture Soundtrack)
 new PUTE for KIDS SEE GHOSTS
 new PUTE for Mozart: Piano Sonatas
 new PUTE for Huncho Jack, Jack Huncho
 new PUTE for Double Or Nothing (& Metro Boomin)
 new PUTE for Live At The Checkerboard Lounge
 new PUTE for Lamomali
 new PUTE for Elgar: Cello Concerto
 new PUTE for Chopin: Piano Concertos & Works for Piano Solo
 new PUTE for Watch The Throne (Deluxe)
 new PUTE for Morale 2
 new PUTE for Beethoven: Violin Concerto; Violin Romances Nos.1 & 2
 new PUTE for Beethoven: The Complete Symphonies and Overtures
 new PUTE for Ella & Louis (

,Artists,Albums
0,Khruangbin,Mordechai
1,Massimo Pericolo,Scialla Semper
2,James Blake,Friends That Break Your Heart
3,Little Simz,Sometimes I Might Be Introvert
4,Django,ATHANOR
...,...,...
380,Jazzy Bazz,P-Town
381,Mac Miller,The Divine Feminine
382,A$AP Rocky,AT.LONG.LAST.A$AP
383,Frank Ocean,Blonde


# Deezer

launch the follwing in the console to get token:<br><br>
*python oauth_deezer.py --app-id 521242 --app-secret 1b2d32133792c353a555ff83c31ac6d8*<br>
*source .env*

<br><br>

## using python Client
### Search track ID for spot library

In [12]:
import deezer
## LOAD ENVIRONMENT VARIABLES : 
from dotenv import load_dotenv
import os
load_dotenv()

#client = deezer.Client(access_token=os.environ['DEEZER_TOKEN'],app_id=os.environ['DEEZER_APP_ID'],app_secret=os.environ['DEEZER_CLIENT_SECRET'])

DEEZER_TOKEN='fr6RFLTmfWeld1pHhSSCLEqUWburRilOqUxJAOjC0hZ6YWY7200'
client = deezer.Client(access_token=DEEZER_TOKEN,app_id=os.environ['DEEZER_APP_ID'],app_secret=os.environ['DEEZER_CLIENT_SECRET'])


In [41]:
def search_deezertracksID_from_spotify_library(df_library_spotify,client,index_to_ban=None):
    """ Takes a Dataframe of Songs and give the Deezer trackID
        Input:  - dataframe with columns:'Artists', 'Tracks', 'Albums', 'TrackID'
                - Deezer client
        Return :    -Tracks_id: list of Track id 
                    -unmatched: list of soungs characteristics unmatched
                    -song_found_without_artist : int
    """
    # get deezer trackID from spotify library:
    Tracks_id = [] # UPDATE THE DATAFRAME DF_library_spotify instead of using a list 
    unmatched=[]
    song_found_without_artist = 0
    df_library_spotify['Deezer_trackid'] = 0
    for index,row in df_library_spotify.iterrows():
        print(str(index))
        if index in index_to_ban : # exception due to format search futher
            unmatched.append([row.Artists,row.Tracks,row.Albums])
            print('unmatched\n')
        else:
            results = client.search(artist=row.Artists,track=row.Tracks,album=row.Albums)
            if len(results)>0:
                Tracks_id.append(results[0].id)
                df_library_spotify['Deezer_trackid'][index] = results[0].id
            else :
                results = client.search(artist=row.Artists,track=row.Tracks)
                if len(results)>0:
                    Tracks_id.append(results[0].id)
                    df_library_spotify['Deezer_trackid'][index] = results[0].id
                else :
                    results = client.search(track=row.Tracks,album=row.Albums)
                    if len(results)>0:
                        Tracks_id.append(results[0].id)
                        df_library_spotify['Deezer_trackid'][index] = results[0].id
                        song_found_without_artist = song_found_without_artist +1           
                    else:
                        unmatched.append([row.Artists,row.Tracks,row.Albums])
    return df_library_spotify,Tracks_id,unmatched,song_found_without_artist

In [113]:
import requests
from dotenv import load_dotenv
import os

def request_json(URL,param_session):
    #session = requests.session()
    #response = session.request("GET",URL,params=param_session)
    response = requests.get(URL,params=param_session)

    try:
        response.raise_for_status()
    except requests.exceptions.HTTPError as e:
        # Whoops it wasn't a 200
        return "Error: " + str(e)

    # Must have been a 200 status code
    json_obj = response.json()
    return json_obj

def deezer_search(artist=None,track=None,album=None):
    #initiate parameters
    URL = 'https://api.deezer.com/search?q='
    load_dotenv()
    param_session = {'app_secret':os.environ['DEEZER_CLIENT_SECRET'],
                'app_id': os.environ['DEEZER_APP_ID'],
                'access_token':'frWCKCQyzmjnisjI1lmQRsjHrgMADK55FdosZx7J300AB7SRakC'#os.environ['DEEZER_TOKEN']
                }
    #launch request
    if artist : 
        URL = URL + (f'artist:"{artist}" ')
    if track : 
        URL = URL + (f'track:"{track}" ')
    if album : 
        URL = URL + (f'album:"{album}" ')

    json_response = request_json(URL,param_session)

    if json_response['total'] > 0 :
        return json_response['data'][0]['id']
    else: 
        return -1
        
def reformat_track_name (trackname):
    if 'feat.' in trackname or '(&'in trackname :
        index = trackname.find('(')
        newname = trackname[:index]
        return newname

def search_deezertracksID_from_spotify_library(df_library_spotify,index_to_ban=[]):
    """ Takes a Dataframe of Songs and give the Deezer trackID
        Input:  - dataframe with columns:'Artists', 'Tracks', 'Albums', 'TrackID'
                - Deezer client
        Return :    -Tracks_id: list of Track id 
                    -unmatched: list of soungs characteristics unmatched
                    -song_found_without_artist : int
    """
    # get deezer trackID from spotify library:
    Tracks_id = [] 
    unmatched=[]
    song_found_without_artist = 0
    df_library_spotify['Deezer_trackid'] = -1
    for index,row in df_library_spotify.iterrows():
        print(str(index))
        if index in index_to_ban : # exception due to format search futher
            unmatched.append([row.Artists,row.Tracks,row.Albums])
            print('unmatched\n')
        else:
            resultID = deezer_search(artist=row.Artists,track=reformat_track_name(row.Tracks),album=row.Albums)
            if resultID>-1:
                Tracks_id.append(resultID)
                #df_library_spotify['Deezer_trackid'][index] = resultID
                df_library_spotify.loc[index,'Deezer_trackid']=resultID
            else :
                resultID = deezer_search(artist=row.Artists,track=reformat_track_name(row.Tracks))
                if resultID>-1:
                    Tracks_id.append(resultID)
                    df_library_spotify.loc[index,'Deezer_trackid']=resultID
                else :
                    resultID = deezer_search(track=reformat_track_name(row.Tracks),album=row.Albums)
                    if resultID>-1:
                        Tracks_id.append(resultID)
                        df_library_spotify.loc[index,'Deezer_trackid']=resultID
                        song_found_without_artist += +1           
                    else:
                        unmatched.append([row.Artists,row.Tracks,row.Albums])
    return df_library_spotify,Tracks_id,unmatched,song_found_without_artist

In [114]:
#index_to_ban = [134,1477,1906,4052]#[128,1474,1903,4049,131] #

df_library_spotify,Tracks_id,unmatched,song_found_without_artist = search_deezertracksID_from_spotify_library(df_library_spotify)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [139]:
len(df_library_spotify[df_library_spotify.Deezer_trackid>-1])

4551

In [146]:
import requests
from dotenv import load_dotenv
import os

"""def request_json(URL,param_session):
    #session = requests.session()
    #response = session.request("GET",URL,params=param_session)
    response = requests.get(URL,params=param_session)
    
    try:
        response.raise_for_status()
    except requests.exceptions.HTTPError as e:
        # Whoops it wasn't a 200
        return "Error: " + str(e)

    # Must have been a 200 status code
    if response.json().get('error') : 
        print(response.json()['error']['message'])
    else:
        json_obj = response.json()
        return json_obj"""

def request_json(operation,URL,param_session):
    session = requests.session()
    response = session.request(operation,URL,params=param_session)
    
    try:
        response.raise_for_status()
    except requests.exceptions.HTTPError as e:
        # Whoops it wasn't a 200
        return "Error: " + str(e)

    # Must have been a 200 status code
    if response.json().get('error') : 
        print(response.json()['error']['message'])
    else:
        json_obj = response.json()
        return json_obj

def deezer_search(artist=None,track=None,album=None):
    #initiate parameters
    URL = 'https://api.deezer.com/search?q='
    load_dotenv()
    param_session = {'app_secret':os.environ['DEEZER_CLIENT_SECRET'],
                'app_id': os.environ['DEEZER_APP_ID'],
                'access_token':'freCiD8xDNCM9M2Gv5jOJOqS9DKHwI85rRNCP6A1Oz7KrtijK8W'#os.environ['DEEZER_TOKEN']
                }
    #launch request
    if artist : 
        URL = URL + (f'artist:"{artist}" ')
    if track : 
        URL = URL + (f'track:"{track}" ')
    if album : 
        URL = URL + (f'album:"{album}" ')

    json_response = request_json('GET',URL,param_session)
    #print(json_response)
    if json_response['total'] > 0 :
        return json_response['data'][0]['id']
    else: 
        return -1

r = deezer_search('Arcade Fire','We used to Wait','The suburbs')

r

377386541

In [94]:
if r.json()['total'] > 0 :
    id = (r.json()['data'][0]['id'])


In [54]:
#df_library_spotify.iloc[134] #1477 1906
df_library_spotify.iloc[4052]

Artists                                    Bob Marley & The Wailers
Tracks            Baby We've Got A Date (Rock It Baby) - Origina...
Albums                                Catch A Fire (Deluxe Edition)
TrackID                                      1oNvFKxkuoiEhT6H5piOrF
Deezer_trackid                                                    0
Name: 4052, dtype: object

In [55]:
results = client.search(artist='Bob Marley & The Wailers',track='Baby We\'ve Got A Date - Original Album Version',album='Catch A Fire (Deluxe Edition)')
results

[]

In [47]:
for i in results:
    print(i.album)

<Album: Catch A Fire>
<Album: Catch A Fire>
<Album: Catch A Fire>


In [31]:
string_to_change = 'Savage Time (& Metro Boomin)'

if 'feat.' in string_to_change or '(&'in string_to_change :
    print(string_to_change)
    index = string_to_change.find('(')
    print(string_to_change[:index])
    newstring = string_to_change[:index]

Savage Time (& Metro Boomin)
Savage Time 


In [64]:
string_to_change = 'Savage Time (& Metro Boomin)'

def reformat_track_name (trackname):
    if 'feat.' in trackname or '(&'in trackname :
        index = trackname.find('(')
        newname = trackname[:index]
        return newname

print(string_to_change)
print(reformat_track_name(string_to_change))

Savage Time (& Metro Boomin)
Savage Time 


## Manual Requests

### GEt ID tracks already saved

In [1]:
DEEZER_TOKEN = 'frS8bNLnhP56by75o58zp2yvam0QKJ4ad02HjnDcVRKaF7F41Pp'

In [35]:
import requests

def deezer_show_track(result):
    tracksID_saved= []
    for i,data in enumerate(result):
        tracksID_saved.append(data['id'])
        #print('- track id :'+str(data['id'])+' title :'+(data['title']))
        #print('artise : '+data['artist']['name'])
        #print('album : '+data['album']['title'])
    return tracksID_saved

def get_user_saved_trackID(URL,param_session):
    
    session =requests.session()
    response = session.request("GET",URL,params=param_session)

    tracksID_already_saved = []
    tracksID_already_saved.extend(deezer_show_track(response.json()['data']))

    while response.json().get('next') is not None:
        response = session.request("GET",url=response.json()['next'])
        tracksID_already_saved.extend(deezer_show_track(response.json()['data']))
    return tracksID_already_saved

URL = 'https://api.deezer.com/user/me/tracks'
param_session ={'app_secret':'1b2d32133792c353a555ff83c31ac6d8',
    'app_id':'521242',
    'access_token':DEEZER_TOKEN}
    
tracksID_already_saved = get_user_saved_trackID(URL,param_session)

In [137]:
import requests

def get_saved_tracksID(param_session):
    URL = 'https://api.deezer.com/user/me/tracks'
    session = requests.session()
    response = session.request("GET",URL,params=param_session)
    if response.json().get('error') : 
        print(response.json()['error']['message'])
    else:
        totaltracks = response.json()['total']
        tracksID_already_saved = []
        trackscounter = 0

        while totaltracks > len(tracksID_already_saved):
            for track in response.json()['data']:
                tracksID_already_saved.append(track['id'])
            trackscounter += len(response.json()['data'])
            if trackscounter < totaltracks:
                response = session.request("GET",url=response.json()['next'])
        return tracksID_already_saved




load_dotenv()

param_session ={'app_secret':os.environ['DEEZER_CLIENT_SECRET'],
                'app_id': os.environ['DEEZER_APP_ID'],
                'access_token':'freCiD8xDNCM9M2Gv5jOJOqS9DKHwI85rRNCP6A1Oz7KrtijK8W'}#os.environ['DEEZER_TOKEN']}


alreadysaved_trackid =(get_saved_tracksID(param_session))



In [140]:
len(alreadysaved_trackid)
spot_Tracks_id = Tracks_id


In [ ]:
def get_saved_tracksID(param_session):
    """
    'param_session ={'app_secret':'1b2d32133792c353a555ff83c31ac6d8',
    'app_id':'521242',
    'access_token':DEEZER_TOKEN}
    """
    URL = 'https://api.deezer.com/user/me/tracks'
    session = requests.session()
    response = session.request("GET",URL,params=param_session)
    if response.json().get('error') : 
        print(response.json()['error']['message'])
    else:
        totaltracks = response.json()['total']
        tracksID_already_saved = []
        trackscounter = 0

        while totaltracks > len(tracksID_already_saved):
            for track in response.json()['data']:
                tracksID_already_saved.append(track['id'])
            trackscounter += len(response.json()['data'])
            if trackscounter < totaltracks:
                response = session.request("GET",url=response.json()['next'])
        return tracksID_already_saved

In [5]:
session = requests.session()
param_session ={'app_secret':'1b2d32133792c353a555ff83c31ac6d8',
    'app_id':'521242',
    'access_token':DEEZER_TOKEN}
URL = 'https://api.deezer.com/user/me/tracks'
response = session.request("GET",URL,params=param_session)

response.json()


{'data': [{'id': 2174774,
   'readable': True,
   'title': 'Ces Gens Là',
   'link': 'https://www.deezer.com/track/2174774',
   'duration': 374,
   'rank': 462746,
   'explicit_lyrics': False,
   'explicit_content_lyrics': 0,
   'explicit_content_cover': 2,
   'md5_image': '928dcf51c5e61bdb200cce1eb8145e06',
   'time_add': 1360965268,
   'album': {'id': 217560,
    'title': 'Noir Désir en public',
    'cover': 'https://api.deezer.com/album/217560/image',
    'cover_small': 'https://e-cdns-images.dzcdn.net/images/cover/928dcf51c5e61bdb200cce1eb8145e06/56x56-000000-80-0-0.jpg',
    'cover_medium': 'https://e-cdns-images.dzcdn.net/images/cover/928dcf51c5e61bdb200cce1eb8145e06/250x250-000000-80-0-0.jpg',
    'cover_big': 'https://e-cdns-images.dzcdn.net/images/cover/928dcf51c5e61bdb200cce1eb8145e06/500x500-000000-80-0-0.jpg',
    'cover_xl': 'https://e-cdns-images.dzcdn.net/images/cover/928dcf51c5e61bdb200cce1eb8145e06/1000x1000-000000-80-0-0.jpg',
    'md5_image': '928dcf51c5e61bdb200cce1

In [9]:
response.json()

if 'error' in response.json().keys():
    print('in')

if response.json().get('error'):
    print('error')

### Add tracks

In [ ]:
len(alreadysaved_trackid)
spot_Tracks_id = Tracks_id

In [174]:
def request_json(operation,URL,param_session):
    session = requests.session()
    response = session.request(operation,URL,params=param_session)
    
    try:
        response.raise_for_status()
    except requests.exceptions.HTTPError as e:
        # Whoops it wasn't a 200
        return "Error: " + str(e)

    # Must have been a 200 status code
    if response.json() == True:
        return True
    elif response.json().get('error') : 
        if response.json()['error']['code'] == 801: # track already saved
            return response.json()
        else:
            print(response.json()['error']['message'])
    else:
        json_obj = response.json()
        return json_obj

In [175]:

def add_track_deezer(tracksID_already_saved,newtracks_id):
    URL = 'https://api.deezer.com/user/me/tracks'
    load_dotenv()
    param_session ={'app_secret':os.environ['DEEZER_CLIENT_SECRET'],
                    'app_id': os.environ['DEEZER_APP_ID'],
                    'access_token':'freCiD8xDNCM9M2Gv5jOJOqS9DKHwI85rRNCP6A1Oz7KrtijK8W'}#os.environ['DEEZER_TOKEN']}
    count_new_tracks = 0
    alreadyIN_track = 0

    #counter=0
    for track in newtracks_id :
        if track in tracksID_already_saved:
            alreadyIN_track += 1
        else:
            #add track
            param_session['track_id'] = track
            response = request_json('POST',URL,param_session)
            if response == True : 
                #succesfully added
                count_new_tracks += 1
            elif response['error']['code']== 801:
                #alreadyIN
                alreadyIN_track += 1
                tracksID_already_saved.append(track)
            else :
                print(response)
                break
    return count_new_tracks,alreadyIN_track,tracksID_already_saved
    
count_new_tracks, alreadyIN_track, tracksID_already_saved = add_track_deezer(alreadysaved_trackid,spot_Tracks_id)

KeyboardInterrupt: 

In [152]:
reponse

NameError: name 'reponse' is not defined

### Create Playlist and add tracks

In [11]:
df_playlist_tokeep.PlaylistName.unique()

array(['Your Top Songs 2021', 'Your Top Songs 2020', 'Italia',
       'Your Top Songs 2018', 'Liked from Radio', 'Shs', 'Inspi',
       'Your Top Songs 2017', 'Mes titres Shazam', 'Romantic songs',
       'Jazz', 'Unexpected', 'Rap - Turn Up', 'Sexy songs - R&B'],
      dtype=object)

In [12]:
df_playlist_tokeep.columns

Index(['PlaylistName', 'TrackID', 'tracklist', 'artistlist', 'albumlist'], dtype='object')

In [60]:
import requests

URL = 'https://api.deezer.com/user/me/playlists'
param_session ={'app_secret':'1b2d32133792c353a555ff83c31ac6d8',
    'app_id':'521242',
    'access_token':'frzu4DG8zCc4bBl5wFy4a4SJp1fojwidxdGzgbYWyoAF1pyxusZ'}

# get deezer list of all playlist
session =requests.session()
response = session.request("GET",URL,params=param_session)


deezer_name_playlists_saved = [elem['title']for elem in response.json()['data']]
deezer_id_playlists_saved = [elem['id']for elem in response.json()['data']]
deezer_playlists = dict(zip(deezer_name_playlists_saved, deezer_id_playlists_saved))

print(deezer_playlists)

{'Classique': 1547989661, 'fat hip hop': 1508914711, 'Jazz Vocal': 1548260491, 'Loved tracks': 191781641, 'newplaylist': 10285143642, 'Soul - Funk': 1565791111, 'Unexpected': 10281775062, 'Soul Essentials': 1257789321, 'Jazzy': 25205324}


In [72]:
def request_json(operation,URL,param_session):
    session = requests.session()
    response = session.request(operation,URL,params=param_session)
    
    try:
        response.raise_for_status()
    except requests.exceptions.HTTPError as e:
        # Whoops it wasn't a 200
        return "Error: " + str(e)
    print(response.json())
    # Must have been a 200 status code
    if response.json() == True:
        return True
    elif response.json().get('error') : 
        if response.json()['error']['code'] == 801: # track already saved
            return response.json()
        else:
            print(response.json()['error']['message'])
            return False
    else:
        json_obj = response.json()
        return json_obj

In [73]:
# create new playlist
URL = 'https://api.deezer.com/user/me/playlists'
nameNewplaylist = 'newplaylist'
id_playlist = -1

# if new playlist create id 
if nameNewplaylist not in deezer_playlists.keys():
    param_session['title']=nameNewplaylist

    session =requests.session()
    response = session.request("POST",URL,params=param_session)

    print(response.json())
    id_playlist = response.json()['id'] 
else :
    id_playlist = deezer_playlists[nameNewplaylist]

print(f'id of playlist : {id_playlist}')

#add songs to the playlist
URL = f'https://api.deezer.com/playlist/{id_playlist}/tracks'

param_session['playlist_id'] = id_playlist
param_session['songs'] = [126902923]#,437147192]




print(request_json('POST',URL,param_session))

id of playlist : 10285143642
{'error': {'type': 'ParameterException', 'message': 'This song already exists in this playlist', 'code': 500}}
This song already exists in this playlist
False


In [ ]:
def add_playlists(df_spotify_playlists,deezer_old_playlists_name,param_session,publicplaylist=True,print_loading=False):
    # for ech playlist
    for playlist in df_spotify_playlists.PlaylistName.unique():
        # add playlist name to the parameters
        df_playlist = df_spotify_playlists[df_spotify_playlists.PlaylistName == playlist]
        #check if playlist alredy exist if not create a new one
        if playlist not in deezer_old_playlists_name:
            # search id for all tracks of the playlist # to check and finish from here
            #index_to_ban = [128,1474,1903,4049,131]  
            df_library_spotify,Tracks_id,unmatched,song_found_without_artist = search_deezertracksID_from_spotify_library(param_session,df_playlist,print_loading=False)
                # check if id already in playlist
            param_session['title'] = playlist
            param_session['public']=publicplaylist
                # if not add track to the playlist
            new_track, alreadyIN_track = add_track_deezer('playlists',param_session,Tracks_id)
            if print_loading:
                print(f'In {playlist} : {new_track} new tracks added, {alreadyIN_track} already saved')
        else : 
            print('check if all songs are the same if not add them')

def add_track_deezer(type_add,param_session,newtracks_id,print_loading=False):
    URL = 'https://api.deezer.com/user/me/'+type_add
    count_new_tracks = 0
    alreadyIN_track = 0

    tracksID_already_saved = get_saved_tracksID(type_add,param_session)

    #counter=0
    for track in newtracks_id :
        if track in tracksID_already_saved:
            alreadyIN_track += 1
        else:
            #add track
            param_session['track_id'] = track
            response = request_json('POST',URL,param_session)
            if response == True : 
                #succesfully added
                count_new_tracks += 1
            elif response['error']['code']== 801:
                #alreadyIN
                alreadyIN_track += 1
                tracksID_already_saved.append(track)
            else :
                print(response)
                break
    if print_loading:
        print(f'{count_new_tracks} new tracks added, {alreadyIN_track} already saved')
    return count_new_tracks,alreadyIN_track

def request_json(operation,URL,param_session):
    session = requests.session()
    response = session.request(operation,URL,params=param_session)
    
    try:
        response.raise_for_status()
    except requests.exceptions.HTTPError as e:
        # Whoops it wasn't a 200
        return "Error: " + str(e)

    # Must have been a 200 status code
    if response.json() == True:
        return True
    elif response.json().get('error') : 
        if response.json()['error']['code'] == 801: # track already saved
            return response.json()
        else:
            print(response.json()['error']['message'])
    else:
        json_obj = response.json()
        return json_obj

      

In [8]:
import requests

URL = 'https://api.deezer.com/user/me/playlists'
param_session ={'app_secret':'1b2d32133792c353a555ff83c31ac6d8',
    'app_id':'521242',
    'access_token':'fr5KZarhguXew8mIECR6rf3L7BmMbSrvULl8eB90EztWg2GjtXi'}

# get deezer list of all playlist
session = requests.session()
client = session.request("GET",URL,params=param_session)


deezer_name_playlists_saved = [elem['title'] for elem in client.json()['data']]


# for ech playlist
for playlist in df_playlist_tokeep.PlaylistName.unique():
    #check if playlist alredy exist
    if playlist not in deezer_name_playlists_saved:
        # get playlist name
        param_session['title'] = playlist
        # search id for all tracks of the playlist # to check and finish from here
        #index_to_ban = [128,1474,1903,4049,131]  
        df_library_spotify,Tracks_id,unmatched,song_found_without_artist = search_deezertracksID_from_spotify_library(df_library_spotify,client)
            # check if id already in playlist
        tracksID_already_saved = get_user_saved_trackID(URL,param_session)
            # if not add track to the playlist
        new_track, alreadyIN_track, tracksID_already_saved = add_track_deezer(URL,param_session,tracksID_already_saved)

#check result on deezer

NameError: name 'search_deezertracksID_from_spotify_library' is not defined

In [68]:
new_trackcount,alreadyIN_trackcount=0,0

In [69]:
new_trackcount

0

In [70]:
alreadyIN_trackcount

0

# Old Version 

In [ ]:
# search tracks OLD VERSION

"""# get deezer trackID from spotify library:
Tracks_id = []
unmatched=[]
for index,row in df_library_spotify.iterrows():
    if index in (128,1471,1901,4046) :
        unmatched.append([row.Artists,row.Tracks,row.Albums])
        print('unmatched\n')
    else:
        results = client.search(artist=row.Artists,track=row.Tracks,album=row.Albums)
        if len(results)>0:
            Tracks_id.append(results[0].id)
        else :
            unmatched.append([row.Artists,row.Tracks,row.Albums])
# with all unmatched launch new algorithm without the album name

Tracks_id_v2 = []
unmatched_v2 = []
song_album = 0

for index,track in enumerate(unmatched):
    artist_ = track[0]
    song_ = track[1]
    album_ = track[2]
    if  index in ([11,121,189,489]) : # Exception due to format, add index to pass
        unmatched_v2.append([artist_,song_,album_])
        print('unmatched\n')
        pass
    else:
        results = client.search(artist=artist_,track=song_)
        if len(results)>0:
            Tracks_id_v2.append(results[0].id)
        else :
            results = client.search(track=song_,album=album_)
            if len(results)>0:
                Tracks_id_v2.append(results[0].id)
                song_album = song_album +1           
            else:
                unmatched_v2.append([artist_,song_,album_])

len(unmatched_v2)"""

In [ ]:
"""
# ALREADY SAVED TRACKS OLD VERSION
def deezer_show_track(result):
    for i,data in enumerate(result):
        tracksID_already_saved.append(data['id'])
        #print('- track id :'+str(data['id'])+' title :'+(data['title']))
        #print('artise : '+data['artist']['name'])
        #print('album : '+data['album']['title'])

URL = 'https://api.deezer.com/user/me/tracks'
param_session ={'app_secret':'1b2d32133792c353a555ff83c31ac6d8',
    'app_id':'521242',
    'access_token':DEEZER_TOKEN}
       
tracksID_already_saved = []
response = session.request("GET",URL,params=param_session)
deezer_show_track(response.json()['data'])

while response.json().get('next') is not None:
    response = session.request("GET",url=response.json()['next'])
    deezer_show_track(response.json()['data'])

#response.content()
#response.text()
#response.json()
#response.json().keys()

#print(len(tracksID_already_saved))
#print(response.json()['total'])
"""

In [165]:
"""
# ADD TRACKS OLD VERSION

URL = 'https://api.deezer.com/user/me/tracks'
param_session ={'app_secret':'1b2d32133792c353a555ff83c31ac6d8',
    'app_id':'521242',
    'access_token':DEEZER_TOKEN}
    
new_track = 0
alreadyIN_track = 0
session = requests.session()

counter=0

for track in Tracks_id :
    counter=counter+1
    print(counter)
    if track in tracksID_already_saved:
        alreadyIN_track=alreadyIN_track+1
    else:
        #add track
        param_session ['track_id'] = track
        response = session.request("POST",URL,params=param_session)
        if response.json() == True : 
            #succesfully added
            new_track = new_track + 1
        elif response.json()['error']['code']== 801:
            #alreadyIN
            alreadyIN_track=alreadyIN_track+1
            tracksID_already_saved.append(track)
        else :
            print(response.json())
            break"""

'\n# ADD TRACKS OLD VERSION\n\nURL = \'https://api.deezer.com/user/me/tracks\'\nparam_session ={\'app_secret\':\'1b2d32133792c353a555ff83c31ac6d8\',\n    \'app_id\':\'521242\',\n    \'access_token\':DEEZER_TOKEN}\n    \nnew_track = 0\nalreadyIN_track = 0\nsession = requests.session()\n\ncounter=0\n\nfor track in Tracks_id :\n    counter=counter+1\n    print(counter)\n    if track in tracksID_already_saved:\n        alreadyIN_track=alreadyIN_track+1\n    else:\n        #add track\n        param_session [\'track_id\'] = track\n        response = session.request("POST",URL,params=param_session)\n        if response.json() == True : \n            #succesfully added\n            new_track = new_track + 1\n        elif response.json()[\'error\'][\'code\']== 801:\n            #alreadyIN\n            alreadyIN_track=alreadyIN_track+1\n            tracksID_already_saved.append(track)\n        else :\n            print(response.json())\n            break'